### 注意：请下载train和test数据放在对应目录下，当前目录下只提供图片示例文件

In [1]:
'''
使用keras下载训练好的模型（下载每个模型几十分钟）, 并用模型对提取图片的特征。
训练集25000张图式片， 测试集12500张图片，用三个模型, 如果不使用GPU，约8小时以上。
本例中只使用了几百张图片, 花十几分钟跑一遍整体流程。 想训练所有， 只需增加图片数据即可。
'''

from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *
import h5py
import warnings

warnings.filterwarnings('ignore')

def get_features(MODEL, width, height, lambda_func=None):
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)

    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    # 注意 train 和 test 是图片存储路径
    train_generator = gen.flow_from_directory("train", (width, height), shuffle=False,
                                              batch_size=16)
    test_generator = gen.flow_from_directory("test", (width, height), shuffle=False,
                                             batch_size=16, class_mode=None)
    # 不同的keras版本变量名略有不同，如: train_generator.samples train_generator.nb_sample...
    train = model.predict_generator(train_generator, train_generator.nb_sample)
    test = model.predict_generator(test_generator, test_generator.nb_sample)
    with h5py.File("data_%s.h5"%MODEL.func_name) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

get_features(ResNet50, 224, 224)
get_features(InceptionV3, 299, 299, inception_v3.preprocess_input)
get_features(Xception, 299, 299, xception.preprocess_input)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# 训练模型并预测，此处使用了深度学习模型，也可以换成机器学习模型

import h5py
import numpy as np
from sklearn.utils import shuffle
from keras.models import *
from keras.layers import *

np.random.seed(12345678)
X_train = []
X_test = []

for filename in ["data_ResNet50.h5", "data_Xception.h5", "data_InceptionV3.h5"]:
    with h5py.File(filename, 'r') as h:
        X_train.append(np.array(h['train']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['label'])

X_train = np.concatenate(X_train, axis=1)
X_test = np.concatenate(X_test, axis=1)
X_train, y_train = shuffle(X_train, y_train)

input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=128, nb_epoch=8, validation_split=0.2)
y_pred = model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

In [ ]:
# 用迭代训练过程中的错误率做图

import matplotlib.pyplot as plt
%matplotlib inline

def plot_training(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    epochs = range(len(acc))
    plt.plot(epochs, acc, 'b')
    plt.plot(epochs, val_acc, 'r')
    plt.legend(["acc", "val_acc"], loc='best')
    plt.title('Training and validation accuracy')
    plt.show()
    loss = history.history['loss']
    val_loss = history.history['val_loss']   
    plt.plot(epochs, loss, 'b')
    plt.plot(epochs, val_loss, 'r')
    plt.legend(["loss", "val_loss"], loc='best')
    plt.title('Training and validation loss')
    plt.show()

plot_training(history)